In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import json
import random,time
from random import randint
import re
import urllib
import sys
from urllib import request
from datetime import datetime
import pandas as pd
from pandas.core.frame import DataFrame
from datetime import datetime

def create_mkdir():
    ####建立資料夾
    mydir='cookpad'
    mydir_save_img="cookpad_save_img"
    mydir_save_log='cookpad_log'
    mydir_save_title_log='cookpad_title_log'
    mydir_save_count_log='cookpad_save_count_log'
    if os.path.exists(mydir):
        print('mrdir name cookpad has been create')
    else:
        os.mkdir(mydir)

    if os.path.exists(mydir_save_img):
        print("mrdir name cookpad_save_img has been create")
    else:
        os.mkdir(mydir_save_img)
    
    if os.path.exists(mydir_save_log):
        print("mrdir name cookpad_save_log has been create")
    else:
        os.mkdir(mydir_save_log)
    if os.path.exists(mydir_save_title_log):
        print('mydir name cookpad_title_log has been create')
    else:
        os.mkdir(mydir_save_title_log)
    if os.path.exists(mydir_save_count_log):
        print('mydir name cookpad_save_count_log has been create')
    else:
        os.mkdir( mydir_save_count_log)
#######

#####寫入檔案相關
def write_to_file(write_file_list):
    with open('./cookpad/'+save_name+'.txt','w',encoding="utf-8") as es:
        es.write(title_name+'|'+'\n')
        es.write(arthor+'|'+'\n')
        es.write(img+'|'+'\n')
        es.write(Number_of_copies+'|'+'\n')
        es.write(cook_time+'|'+'\n')
        es.write(set_food+'|'+'\n')
        es.write(set_food_count+'|'+'\n')
        es.write(set_step+'|'+'\n')
        print('寫檔完成')

def save_img(img):
    if img!="na":
        print("進入寫入圖片前")
        request.urlretrieve(img, './cookpad_save_img/{}.jpg'.format(save_name))#寫入圖片
        print("圖片下載完成")
    else:
        print("image not found")
def save_log(set_next_url):
    with open('./cookpad_log/'+str(set_next_url)+'_log'+'.txt','w',encoding="utf-8") as es2:
        es2.write(getnextpageurl)
    
def save_title_log(set_title_log_url):
    with open('./cookpad_title_log/'+str(set_title_log_url)+"_title_log"+'.txt','w',encoding='utf-8')as es3:
        es3.write(title_url)
def save_count_log(set_total_items):
    with open('./cookpad_save_count_log/'+str(set_total_items)+'_count_log'+'.txt','w',encoding='utf-8')as es4:
        es4.write(str(set_total_items))

def list_to_csv(list_save):
    data=DataFrame(list_save)#这时候是以行为标准写入的
    print(data)
    now = datetime.now()
    s1 = datetime.strftime(now,'%Y-%m-%d')
    s2= datetime.strftime(now,'%H:%M:%S')
    s2=s2.replace(":","_")
    #print (type(now))
    #print (now)
    #print (s1)
    #print(s2)
    save_file_data_time=s1+"_"+s2
    #print(s.replace(" ","")))
    data.to_csv('cookpad{}.csv'.format(save_file_data_time),header=False,index=False,encoding='utf-8-sig')        

######## 


####文章控制相關
def count_time():
    time_strat=time.time()
    #time.sleep(5)
    time.sleep(randint(5,15))
    time_end=time.time()
    print(time_end-time_strat)


    
def random_headers():
    headers_list = [{'User-agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0'},{'User-agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'},{'User-agent':'Mozilla/5.0 (compatible; MSIE 9.0; Windows Phone OS 7.5; Trident/5.0; IEMobile/9.0)'}]
    headers=random.choice(headers_list)
    print(headers)
    return headers
    

    
def get_main_body(url,headers):
   
    ss=requests.session()
    res=ss.get(url,headers=headers)
   
    res_status_code=res.status_code
    
    if res_status_code!=200:
        proxy_list=[{'https':'59.124.224.180:3128','http':'59.124.224.180:3128'},{'https':'113.196.140.162:8888','http':'113.196.140.162:8888'}]
        #proxy_list2={'http':'http://220.135.165.38:8080','https':'220.135.168.38:8080'}
        proxy=random.choice(proxy_list)
        
        print("進入proxy狀態,ip已經被鎖定")       
        cookies=ss.cookies.get_dict()
        time.sleep(2)
        
        res=ss.get(url,headers=headers,proxies=proxy ,cookies=cookies)
        soup=BeautifulSoup(res.text,'html.parser')
        return soup
            
            #print(res.json())    
    else:
        cookies=ss.cookies.get_dict()
        #print(ss.cookies.get_dict())
        time.sleep(2)
        res=ss.get(url,headers=headers,cookies=cookies)
        soup=BeautifulSoup(res.text,'html.parser')
        return soup


    
    #print(proxy)
#食材,食材數量,步驟

def setfood_and_set_food_count_step(food_test):
    set_food=''
    set_step=''
    set_food_count=''
    step_number=1
    for x in food_test:
        get_food=x.select('div')[0].text
        #print(get_food)
        check_len=len(get_food.split(' '))
        #print(check_len)
        try:
            get_food=get_food.split(' ')[8]
            set_food=set_food+get_food+' '
        except(IndexError):
            print('get_food陣列沒有第8個值')
        #print(get_food)
        get_food_count=x.select('div')[0].select('bdi')[0].text
    
        set_food_count=set_food_count+get_food_count+" "
    for z in step:
        get_step=z.text
        set_step=set_step+str(step_number)+"."+get_step
        step_number=step_number+1

    return set_food, set_food_count  ,set_step      

    
def check_title_log_url(title_url):
    count_wrong=5
    save_list=[]   
    #建立檢查的list
    #test='./cookpad_title_log_check/1_泰式檸檬魚簡易版.txt'
    test_list=os.listdir('./cookpad_title_log/')
    #print(test_list)
    for i in test_list:
        with open ('./cookpad_title_log/'+i,'r',encoding='utf-8')as es4:
            data=es4.read()
        #print(data)
            save_list.append(data)
    
    for g in save_list:
        if g==title_url:
           count_wrong=6
     
           
    if count_wrong==6:
        print("這個url已經重複了,程式不寫入檔案" )
        return count_wrong
    else:
        print("這個url沒有重複,程式繼續")
        return count_wrong
def check_set_total_items():
    save_list=[]   
    #建立檢查的list
    #test='./cookpad_title_log_check/1_泰式檸檬魚簡易版.txt'
    test_list=os.listdir('./cookpad_save_count_log/')
    #print(test_list)
    for i in test_list:
        with open ('./cookpad_save_count_log/'+i,'r',encoding='utf-8')as es4:
            data=es4.read()
        #print(data)
            save_list.append(data)
    set_total_items=int(len(save_list))+1
    return set_total_items

######

create_mkdir()

###一些計數器變數    
headers={'User-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0'}
url="https://cookpad.com/tw?after=1592711907.0" #從首頁開始爬是https://cookpad.com/tw after 後參數為linux_time 在轉換為現在時間
set_next_url_count=1
set_next_url=1     #nextlogurl編號
set_total_items=check_set_total_items() #檔案編號
set_while_stop=1
list_save=[['食譜名稱', '作者','圖片','份數', '料理時間', '食材','食材數量', '料理步驟']]
tep_list=[]
save_next_html=[]#存下檢查爬到哪裡了

######

while (set_while_stop==1):
#for go  in  range(1,3):
  
    headers=random_headers()
    soup=get_main_body(url,headers)
    #print(soup)    
    mainbody=soup.select('div.media')
    getnextpageurl="https://cookpad.com"+soup.select('div#feed_pagination')[0].select('a')[0]['href']
    save_next_html.append(getnextpageurl)#紀錄html
    

    for i in mainbody:
        
        print("第{}筆資料".format(set_total_items))
        title=i.select('div.media__body')[0].select('a.flex-grow')[0]['aria-label']
        test2=i.select('div.media__body')[0].select('a.flex-grow')[0]
        title_url="https://cookpad.com"+i.select('div.media__body')[0].select('a.flex-grow')[0]['href']
        url=title_url
     
        headers=random_headers()
        soup=get_main_body(url,headers)
        #print(soup)
    
        print(title)
        #print(test2)
        print(title_url)
        
        count_time()
        #如果img沒有抓到
        img=soup.select('div.tofu_image')
        if len(img)<=0:
            print("img is not found")
            img="na"
        else:
            img=img[0].select('img')[0]['src']
            split_img=img.split(':')[1]#進行切片防止:被轉碼
            test=urllib.parse.quote(split_img)#網址中文轉碼
            img="https:"+test#進行網址重組
        title_name=soup.select('section.intro-container')[0].select('h1.break-words')[0].text
        
        #artice=soup.select('div.recipe-show__story')[0].select('p.mb-2')[0].text.split('#')[0]
        
        arthor=soup.select('section.document__section.media.author-container')[0].select('span.text-cookpad-16')[0].text
        cook_time=soup.select('div.p-3.border-t.border-quaternary.text-center.text-secondary')
        if len(cook_time) <= 0 :
            cook_time = "Na"
            print("cook_tine is na")
        else:
            cook_time = cook_time[0].select('span.px-4')[0].text #有些資料沒有
           
        food_test=soup.select('div.ingredient-list')[0].select('ol.list-unstyled.text-cookpad-18')[0].select('li')
        Number_of_copies=soup.select('div.text-secondary.mt-2')[0].text
        if len(Number_of_copies.split())<=0:
            Number_of_copies='na'
        else:
            print('Number_of_copies is alive')
    
            
        step=soup.select('ol.numbered-list')[0].select('p.mb-2')
            
        set_food, set_food_count ,set_step= setfood_and_set_food_count_step(food_test)   
       
        
        step_number=1#重設步驟變數
        #food_total_and_food_step(food_test,step,step_number)
        
        #print(img)
        #print(title_name)
        #print(artice)
        #print(Awards)
        #print(arthor)
        #print(cook_time)
        #print(food_total)
        #print(set_food)
        #print(set_step)
        #print(img)
        #print(Number_of_copies)
        #存入資料
       
        
        #print(list_save)
        tep_list=[]
        # write_to_file(img, content, cook, cook_time)
           
        #print(list_save)
        #print("_____________________________________")
            
        #print("正規化檔名前")
        #print("_____________________________________")
        string=title_name
        re_title_name=re.sub(u"([^\u4e00-\u9fa5\u0030-\u0039\u0041-\u005a\u0061-\u007a])","",string)
        #print(re_title_name)
        save_name_id=str(set_total_items)
        save_name=save_name_id+"_"+re_title_name
        
        title_name=title_name.replace(" ","")
        set_food=set_food.replace(" ","")
            
        tep_list.append(title_name)
        tep_list.append(arthor)
        tep_list.append(img)
        tep_list.append(Number_of_copies)
        tep_list.append(cook_time)
        tep_list.append(set_food)
        tep_list.append(set_food_count)
        tep_list.append(set_step)
        list_save.append(tep_list)
        
        
        count_wrong=check_title_log_url(title_url)
        
        if count_wrong!=6:
            print("程式繼續")
        else:
            set_total_items+=1
            print("程式不寫檔")
            continue
            
        
        
        #print(save_name)
        #print("正規化檔名後")
            
            
            
            
        #print("_____________________________________")
        
        #print(入寫檔前')
        
        write_file_list=[title_name, arthor, img, Number_of_copies,cook_time,set_food,set_food_count,set_step]
        write_to_file(write_file_list)
        save_title_log(save_name)    
        save_img(img)    
        save_count_log(set_total_items)
                
           
            
        print("_____________________________________")
            
        
      
        set_total_items+=1

    #寫入next_URL_LOG       
    save_log(set_next_url)
    set_next_url+=1
    
    url=getnextpageurl
    print("+++++++換頁+++++++++++++++++")
    print("第{}個頁面".format(set_next_url_count))
    
    print(getnextpageurl)
    set_next_url_count+=1
"""
    if set_total_items==1001:
        set_while_stop=set_while_stop-1
        print("已經有一千筆資料了")  
"""    

if getnextpageurl=='https://cookpad.com/tw?after=1526305209.0':
    set_while_stop=set_while_stop-1
    
    

print(set_total_items-1)

list_to_csv(list_save)




mrdir name cookpad has been create
mrdir name cookpad_save_img has been create
mrdir name cookpad_save_log has been create
mydir name cookpad_title_log has been create
mydir name cookpad_save_count_log has been create
{'User-agent': 'Mozilla/5.0 (compatible; MSIE 9.0; Windows Phone OS 7.5; Trident/5.0; IEMobile/9.0)'}
第77筆資料
{'User-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}
消暑飲 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12929121-%E6%B6%88%E6%9A%91%E9%A3%B2
11.0
cook_tine is na
這個url已經重複了,程式不寫入檔案
程式不寫檔
第78筆資料
{'User-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}
西式高湯｜純天然, 無添加, 無味精, 完全自家制雞湯
(附影片) link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12944483-%E8%A5%BF%E5%BC%8F%E9%AB%98%E6%B9%AF%EF%BD%9C%E7%B4%94%E5%A4%A9%E7%84%B6-%E7%84%A1%E6%B7%BB%E5%8A%A0-%E7%84%A1%E5%91%B3%E7%B2%BE-%E5%AE%8C%E5%85%A8%E8%87%AA%E5%AE%B6%E5%88%B6%E

8.0
cook_tine is na
這個url已經重複了,程式不寫入檔案
程式不寫檔
第102筆資料
{'User-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}
蕃茄燜牛腩牛筋 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12944806-%E8%95%83%E8%8C%84%E7%87%9C%E7%89%9B%E8%85%A9%E7%89%9B%E7%AD%8B
9.0
Number_of_copies is alive
這個url已經重複了,程式不寫入檔案
程式不寫檔
第103筆資料
{'User-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}
蔥花蛋（快速蛋料理） link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12944605-%E8%94%A5%E8%8A%B1%E8%9B%8B%E5%BF%AB%E9%80%9F%E8%9B%8B%E6%96%99%E7%90%86
8.0
cook_tine is na
這個url已經重複了,程式不寫入檔案
程式不寫檔
第104筆資料
{'User-agent': 'Mozilla/5.0 (compatible; MSIE 9.0; Windows Phone OS 7.5; Trident/5.0; IEMobile/9.0)'}
高麗菜飯佐金沙蘑菇嫩雞胸 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12944129-%E9%AB%98%E9%BA%97%E8%8F%9C%E9%A3%AF%E4%BD%90%E9%87%91%E6%B2%99%E8%98%91%E8%8F%87%E5%AB%A9%E9%9B%9E%E8%83%B8
9.0
Number_of_cop

簡單的紅燒肉 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12941551-%E7%B0%A1%E5%96%AE%E7%9A%84%E7%B4%85%E7%87%92%E8%82%89
10.0
cook_tine is na
Number_of_copies is alive
這個url已經重複了,程式不寫入檔案
程式不寫檔
第128筆資料
{'User-agent': 'Mozilla/5.0 (compatible; MSIE 9.0; Windows Phone OS 7.5; Trident/5.0; IEMobile/9.0)'}
低碳Dalgano咖啡 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12889264-%E4%BD%8E%E7%A2%B3dalgano%E5%92%96%E5%95%A1
9.0
cook_tine is na
Number_of_copies is alive
這個url已經重複了,程式不寫入檔案
程式不寫檔
第129筆資料
{'User-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0'}
鹼粽 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12941561-%E9%B9%BC%E7%B2%BD
10.0
Number_of_copies is alive
這個url已經重複了,程式不寫入檔案
程式不寫檔
第130筆資料
{'User-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}
蕃茄煮冬瓜 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12941401-%E8%95%83%E8%8C%84%E7%85%AE%E5%86%AC%E7%93%9C
7.0
cook_tine is na
這個url已經重複了

9.000999927520752
cook_tine is na
這個url沒有重複,程式繼續
程式繼續
寫檔完成
進入寫入圖片前
圖片下載完成
_____________________________________
第154筆資料
{'User-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}
醜豆炒香菇 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12921519-%E9%86%9C%E8%B1%86%E7%82%92%E9%A6%99%E8%8F%87
15.0
Number_of_copies is alive
這個url沒有重複,程式繼續
程式繼續
寫檔完成
進入寫入圖片前
圖片下載完成
_____________________________________
第155筆資料
{'User-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0'}
豬肉腸蔥花煎蛋 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12879095-%E8%B1%AC%E8%82%89%E8%85%B8%E8%94%A5%E8%8A%B1%E7%85%8E%E8%9B%8B
13.0
Number_of_copies is alive
這個url沒有重複,程式繼續
程式繼續
寫檔完成
進入寫入圖片前
圖片下載完成
_____________________________________
第156筆資料
{'User-agent': 'Mozilla/5.0 (compatible; MSIE 9.0; Windows Phone OS 7.5; Trident/5.0; IEMobile/9.0)'}
檸檬漬豆薯 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12932819-%E6%AA%B8%E6%AA%A

In [ ]:
#print(save_next_html)
print(len(save_next_html))




In [ ]:
#print(list_save)
print(len(list_save))
print(list_save[990])



In [ ]:
import pandas as pd
from pandas.core.frame import DataFrame
from datetime import datetime
#包含两个不同的子列表[1,2,3,4]和[5,6,7,8]
data=DataFrame(list_save)#这时候是以行为标准写入的
print(data)

now = datetime.now()
s1 = datetime.strftime(now,'%Y-%m-%d')
s2= datetime.strftime(now,'%H:%M:%S')
s2=s2.replace(":","_")
#print (type(now))
#print (now)
print (s1)
print(s2)
save_file_data_time=s1+"_"+s2
#print(s.replace(" ","")))

data.to_csv('cookpad{}.csv'.format(save_file_data_time),header=False,index=False,encoding='utf-8-sig')

In [ ]:
!pip install pandas

In [ ]:
set_total_items=2


In [ ]:
def parseProxyIpList(content):
    list = []

    soup = BeautifulSoup(content, 'html.parser')
    ips = soup.findAll('tr')
    for x in range(1, len(ips)):
        tds = ips[x].findAll('td')
        ip_temp = 'http://' + tds[1].contents[0] + ':' + tds[2].contents[0]
        print('發現ip：%s' % ip_temp)
        list.append(ip_temp)
    return list

In [ ]:
#代理伺服器網站:http://www.freeproxylists.net/zh/?c=TW&pt=&pr=&a%5B%5D=0&a%5B%5D=1&a%5B%5D=2&u=0 or 
import requests
import random,time
proxy_list=[{'https':'59.124.224.180:3128'},{'https':'https://113.196.140.162:8888'}]
proxy_list2={'http':'http://220.135.165.38'}
proxy=random.choice(proxy_list)
print(proxy)



In [ ]:
def validateIp(proxy):
    proxy_list=[{'https':'59.124.224.180:3128','http':'59.124.224.180:3128'},{'https':'113.196.140.162:8888','http':'113.196.140.162:8888'}]
    proxy_list2={'http':'http://220.135.165.38:8080','https':'220.135.168.38:8080'}
    proxy=random.choice(proxy_list)
    #print(proxy)
    url="https://cookpad.com/tw?after=1592711907.0"
    headers={'User-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0'}
    #proxies=proxy  
    print(proxy)
    try:
        res = requests.get(url ,proxies=proxy ,headers=headers)
        print(res.status_code)
        #print(res.text)
        #print(res.json())    
   
    except Exception as e:
        proxy=proxy_list2
        res = requests.get(url ,proxies=proxy ,headers=headers)
        print("這是備源proxy")
   

test=validateIp(proxy)
print(test)
    

In [ ]:
{'origin': '111.249.136.44'}